In [28]:
import random
from threading import Thread, Event

In [26]:
class Card(object):
    def __init__(self):
        self.text = ""

In [ ]:
class Cards(object):
    pass

In [14]:
class GameException(Exception):
    pass

class PlayerException(Exception):
    pass

In [27]:
class Player(object):
    
    def __init__(self):
        self.mana = 0
        self.hero = None
        self.hero_power = None
        self.deck = []
        self.hand = []
        
    def check_ready(self):
        if len(self.deck)<30:
            raise PlayerException("Player has less than 30 cards")
    
    def increase_mana(self):
        if self.mana == 10: return
        self.mana += 1
        
    def draw_card(self):
        new_card = self.deck.pop()
        self.hand.append(new_card)
        
    def first_draw(self,n):
        self.current_draw = [self.deck.pop() for _ in range(n)]
        return self.current_draw()
        

In [16]:
class Game(object):
    def __init__(self):
        self.p1 = None
        self.p2 = None
        self.turn = 0
        self.current_player = p1
        self.started = False
        self.actions = []
        self.execution_thread = Thread(target=self.run)
        self.execution_thread.daemon = True
        
    def get_action(self,choices):
        
    def add_player(self,player):
        player.check_ready()
        if not self.p1:
            self.p1 = player
            return
        if not self.p2:
            self.p2 = player
            return
        raise GameException("There are already 2 players")
        
    def start(self):
        self.execution_thread.start()
        
    def wait_for_action(self):
        self.pause_event = Event()
        self.pause_event.wait()
    
    def resume(self):
        self.pause_event.set()
        
    def run(self):
        self.started = True
        self.init_player()
        self.init_next_turn()
        
    def init_player(self):
        n_cards = 3 if self.current_player == self.p1 else 4
        gets_coin = self.current_player == self.p2
        draw = self.current_player.first_draw()
        actions = self.get_actions()
        
        
    def switch_player(self):
        self.current_player = self.p1 if self.current_player == self.p2 else self.p2
        
        
    def init_next_turn(self):
        self.current_player.increase_mana()
        self.current_player.draw_card()
    
    def send_action(self):
        
    def end_turn(self):
        pass
        
    def get_actions(self):
        

In [17]:
g = Game()

In [24]:
p1 = Player()
p1.deck = [x for x in range(30)]

In [25]:
g.add_player(p1)

GameException: There are already 2 players

In [29]:
t = Thread()